# Cardiometabolic Risk: SSL Pretraining on Colab

**Phase 5**: Train a self-supervised PPG encoder on 4,133 signals using Colab T4 GPU

**Expected runtime**: 8–12 hours (50 epochs)  
**Output**: Pretrained encoder checkpoint + training metrics

**Prerequisites**:
- Data uploaded to Google Drive: `/MyDrive/cardiometabolic-risk-colab/data/processed/`
- GitHub repo exists and is public

---

## Setup: Mount Drive & Clone Repo

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

from pathlib import Path
COLAB_DRIVE_PATH = Path('/content/drive/MyDrive/cardiometabolic-risk-colab')
print(f"✅ Drive mounted: {COLAB_DRIVE_PATH.exists()}")

In [ ]:
import subprocess
import os

repo_dir = Path('/content/repo')
repo_url = "https://github.com/Yendoh-Derek/Cardiometabolic-Risk-System-for-Wearables.git"

if not repo_dir.exists():
    print("Cloning repository...")
    subprocess.run(["git", "clone", "--depth", "1", repo_url, str(repo_dir)], check=True)
    print(f"✅ Repo cloned: {repo_dir}")
else:
    print(f"✅ Repo already present: {repo_dir}")

os.chdir(repo_dir)

## Install Dependencies

In [ ]:
!pip install -q -r requirements.txt
print("✅ Dependencies installed")

## Verify GPU & Imports

In [ ]:
# Check GPU
!nvidia-smi --query-gpu=name --format=csv,noheader

import torch
print(f"\n✅ GPU available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"   Device: {torch.cuda.get_device_name(0)}")
    print(f"   Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

In [ ]:
import sys
sys.path.insert(0, str(repo_dir / "colab_src"))

from colab_src.models.ssl.config import SSLConfig
from colab_src.models.ssl.encoder import ResNetEncoder
from colab_src.models.ssl.decoder import ResNetDecoder
from colab_src.models.ssl.losses import SSLLoss

print("✅ All imports successful")

# Load config
cfg = SSLConfig.from_yaml("configs/ssl_pretraining.yaml")
print(f"✅ Config loaded")

## Verify Data Integrity

In [ ]:
import pandas as pd

data_dir = repo_dir / "data" / "processed"

# If in Colab, symlink Drive data to repo structure
try:
    drive_data = COLAB_DRIVE_PATH / "data" / "processed"
    if drive_data.exists() and not data_dir.exists():
        print(f"Linking Drive data: {drive_data} → {data_dir}")
        subprocess.run(["ln", "-s", str(drive_data), str(data_dir)], check=True)
except Exception as e:
    print(f"Warning: {e}")

# Verify required files
required_files = {
    "ssl_pretraining_data.parquet": "Training metadata",
    "ssl_validation_data.parquet": "Validation metadata",
    "denoised_signals": "Ground truth signals (denoised)",
}

print("🔍 Checking data integrity...\n")
all_present = True

for fname, description in required_files.items():
    fpath = data_dir / fname
    if fpath.exists():
        if fpath.is_dir():
            count = len(list(fpath.glob("*.npy")))
            print(f"✅ {fname:40s} ({count:5d} files) — {description}")
        else:
            size_mb = fpath.stat().st_size / 1e6
            print(f"✅ {fname:40s} ({size_mb:6.1f} MB) — {description}")
    else:
        print(f"❌ {fname:40s} NOT FOUND — {description}")
        all_present = False

if not all_present:
    print("\n⚠️  MISSING DATA FILES")
    print("\nTo fix:")
    print("  1. Upload data from your local PC to Google Drive")
    print(f"  2. Path: /MyDrive/cardiometabolic-risk-colab/data/processed/")
    print("  3. Re-run this cell")
    raise FileNotFoundError("Data files not found")

# Verify metadata
train_meta = pd.read_parquet(data_dir / "ssl_pretraining_data.parquet")
print(f"\n✅ Training dataset: {len(train_meta)} samples")

val_meta = pd.read_parquet(data_dir / "ssl_validation_data.parquet")
print(f"✅ Validation dataset: {len(val_meta)} samples")

print("\n" + "="*70)
print("✅ ALL DATA READY FOR TRAINING")
print("="*70)

## Phase 5: Run Full Training (50 Epochs)

In [ ]:
# Create output directory for checkpoints
checkpoint_dir = COLAB_DRIVE_PATH / "phase5_checkpoints"
checkpoint_dir.mkdir(parents=True, exist_ok=True)

print(f"📁 Checkpoints will be saved to:")
print(f"   {checkpoint_dir}")
print(f"\n⏱️  Estimated duration: 8–12 hours")
print(f"💾 Batch size: 8 (with 4× accumulation = eff. 32)")
print(f"🔢 Epochs: 50")
print(f"📊 Training samples: 4,133")
print("\n" + "="*70)
print("Starting training...")
print("="*70)

In [ ]:
# Run training script
cmd = [
    "python",
    "colab_src/models/ssl/train.py",
    "--config", "configs/ssl_pretraining.yaml",
    "--device", "cuda",
    "--num_epochs", "50",
    "--output_dir", str(checkpoint_dir),
]

result = subprocess.run(cmd)
print(f"\n✅ Training complete (exit code: {result.returncode})")

## Validate & Visualize Results

In [ ]:
import json
import matplotlib.pyplot as plt

metrics_file = checkpoint_dir / "training_metrics.json"

if metrics_file.exists():
    with open(metrics_file) as f:
        metrics = json.load(f)
    
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Training loss
    axes[0].plot(metrics['train_loss'], linewidth=2, color='steelblue')
    axes[0].set_xlabel('Epoch', fontsize=12)
    axes[0].set_ylabel('Loss', fontsize=12)
    axes[0].set_title('Training Loss (MSE+SSIM+FFT)', fontsize=14, fontweight='bold')
    axes[0].grid(alpha=0.3)
    
    # Validation loss
    if 'val_loss' in metrics:
        axes[1].plot(metrics['val_loss'], linewidth=2, color='coral')
        axes[1].set_xlabel('Epoch', fontsize=12)
        axes[1].set_ylabel('Loss', fontsize=12)
        axes[1].set_title('Validation Loss', fontsize=14, fontweight='bold')
        axes[1].grid(alpha=0.3)
    
    plt.tight_layout()
    plt.savefig(checkpoint_dir / 'loss_curves.png', dpi=150, bbox_inches='tight')
    plt.show()
    
    print(f"✅ Loss curves plotted and saved")
    print(f"\n📊 Final metrics:")
    print(f"   Train loss: {metrics['train_loss'][-1]:.4f}")
    if 'val_loss' in metrics:
        print(f"   Val loss:   {metrics['val_loss'][-1]:.4f}")
else:
    print(f"⚠️  Metrics file not found: {metrics_file}")

## ✅ Phase 5 Complete

Checkpoints are saved to Google Drive at:
```
/MyDrive/cardiometabolic-risk-colab/phase5_checkpoints/
```

**Next Steps**:
1. Phase 6: Linear probe evaluation
2. Phase 7: Extract embeddings
3. Phase 8: Train XGBoost models

See [README.md](../README.md) for detailed instructions.